In [1]:
import pandas as pd
import folium
from folium.plugins import Fullscreen
import numpy as np

In [2]:
df = pd.read_csv('data/df_translated.csv')

In [3]:
df

,group,family,simplified_species,commum_name,threat_category,acronym_threat_category,presence_in_protected_areas,national_action_plan_for_conservation_,fishing_planning,level_of_protection_in_the_national_strategy,exclusive_species_of_brazil,occurrence_states_y,main_threats_y,biome,category,lats_longs_animals
0,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Agriculture,Atlantic Forest,Invertebrates and Crustaceans,[-41.3009708 -8.65956912]
1,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Pollution,Atlantic Forest,Invertebrates and Crustaceans,[-39.10658269 -16.67170236]
2,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Wildfires,Atlantic Forest,Invertebrates and Crustaceans,[-44.98654659 -14.63625901]
3,Aquatic Invertebrates,Aeglidae,Aegla brevipalma,Egla,Critically Endangered (CR),CR,False,True,False,1,True,SC,Fishing,Atlantic Forest,Arthropods,[-49.33628291 -28.99286816]
4,Aquatic Invertebrates,Aeglidae,Aegla camargoi,Egla,Endangered (EN),EN,False,True,False,1,True,RS,Fishing,Atlantic Forest,Arthropods,[-53.30285658 -32.53115665]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,Marine Fish,Squatinidae,Squatina occulta,Cação-anjo-de-asa-curta,Critically Endangered (CR),CR,True,True,True,2,False,RS,Exotic Species,Marine,Other,[-55.68394313 -28.12078297]
2700,Marine Fish,Squatinidae,Squatina occulta,Cação-anjo-de-asa-curta,Critically Endangered (CR),CR,True,True,True,2,False,SC,Agriculture,Marine,Other,[-51.22716799 -27.64183568]
2701,Marine Fish,Squatinidae,Squatina occulta,Cação-anjo-de-asa-curta,Critically Endangered (CR),CR,True,True,True,2,False,SC,Exotic Species,Marine,Other,[-50.00311261 -29.16706076]
2702,Marine Fish,Squatinidae,Squatina occulta,Cação-anjo-de-asa-curta,Critically Endangered (CR),CR,True,True,True,2,False,SP,Agriculture,Marine,Other,[-44.09472863 -22.65372852]


In [4]:
def get_species_data(species_name):
    return df[df['simplified_species'] == species_name]

def get_atribute(df, atribute):
    if atribute == 'lats_longs_animals':
        string_coords = df[atribute].unique()[0].strip('[]')
        coordinates_str = string_coords.split()
        coordinates = [float(coord) for coord in coordinates_str][::-1]
        return coordinates
    elif len(df[atribute].unique()) >1:
        return df[atribute].unique()
    else:
        return df[atribute].unique()

In [5]:

scientific_name = 'Achirus mucuri'

df_test = get_species_data(scientific_name)
df_test

,group,family,simplified_species,commum_name,threat_category,acronym_threat_category,presence_in_protected_areas,national_action_plan_for_conservation_,fishing_planning,level_of_protection_in_the_national_strategy,exclusive_species_of_brazil,occurrence_states_y,main_threats_y,biome,category,lats_longs_animals
0,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Agriculture,Atlantic Forest,Invertebrates and Crustaceans,[-41.3009708 -8.65956912]
1,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Pollution,Atlantic Forest,Invertebrates and Crustaceans,[-39.10658269 -16.67170236]
2,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,True,False,False,1,True,BA,Wildfires,Atlantic Forest,Invertebrates and Crustaceans,[-44.98654659 -14.63625901]


In [6]:
df_count_biomes = df[['simplified_species', 'biome']]
df_count_biomes = df_count_biomes.drop_duplicates()

In [7]:
df = df.rename(columns={'commum_name':'comum_name'})

colors = ['#581845', '#C70039', '#FF5733', '#FFC300']
dangers = ['PEX', 'CR', 'EN', 'VU']
danger_color = dict(zip(dangers, colors))

In [8]:
def generate_species_info_html(comum_name, simplified_species, group, threat_category,
                                presence_in_protected_areas, national_action_plan_for_conservation_, 
                                fishing_planning, exclusive_species_of_brazil, occurrence_states, main_threats, biomes, acronym_threat_category):
    
    protected_status = "present" if presence_in_protected_areas[0] else "absent"
    conservation_plan = "a" if national_action_plan_for_conservation_[0] else "no"
    fishing_plan = "has" if fishing_planning[0] else "has not"
    exclusive_status = "is" if exclusive_species_of_brazil[0] else "is not"

    # Create a list of colored states and threats
    colored_states = [f"<span style='color: #0066BA'>{state}</span>" for state in np.unique(occurrence_states)]
    colored_threats = [f"<span style='color: red'>{threat}</span>" for threat in  np.unique(main_threats)]
    colored_biome = [f"<span style='color: green'>{biome}</span>" for biome in np.unique(biomes)]
    

    

    

    # Join the colored states and threats with commas and replace the last comma with "and"
    all_biomes = ("biomes of " if len(biomes) > 1 else "biome of ") + ", ".join(colored_biome[:-1]) + (", and " if len(biomes) > 1 else "") + colored_biome[-1]
    all_states = ", ".join(colored_states[:-1]) + (", and " if len(occurrence_states) > 1 else "") + colored_states[-1]
    all_threats = ", ".join(colored_threats[:-1]) + (", and " if len(occurrence_states) > 1 else "") + colored_threats[-1]

    html_text = f"<h1>The {simplified_species}</h1><br>"
    html_text += f"Commonly referred to as {comum_name[0]}, it belongs to the {group[0]} group. "
    html_text += f"This species is categorized as <span style='color: {danger_color[acronym_threat_category[0]]}'>{threat_category[0]}</span> and is {protected_status} in protected areas. "
    html_text += f"There is {conservation_plan} Brazilian conservation action plan in place, and it {fishing_plan} a fishing management strategy. "
    html_text += f"{simplified_species} {exclusive_status} exclusive to Brazil; it can also be found in the states of {all_states}. "
    html_text += f"Its primary threats include {all_threats}."
    html_text += f"This species can be found in the {all_biomes}."
    
    return html_text



def get_markers_tuples(df, simplified_species):
    
    for species in simplified_species:
        try:
            df_specie = get_species_data(species).dropna()
            
            category = get_atribute(df_specie, 'category')
            comum_name = get_atribute(df_specie, 'comum_name')
            group = get_atribute(df_specie, 'group')
            threat_category = get_atribute(df_specie, 'threat_category')
            presence_in_protected_areas = get_atribute(df_specie, 'presence_in_protected_areas')
            national_action_plan_for_conservation_ = get_atribute(df_specie, 'national_action_plan_for_conservation_')
            fishing_planning = get_atribute(df_specie, 'fishing_planning')
            exclusive_species_of_brazil = get_atribute(df_specie, 'exclusive_species_of_brazil')
            occurrence_states = get_atribute(df_specie, 'occurrence_states_y')
            main_threats = get_atribute(df_specie, 'main_threats_y')
            biomes = get_atribute(df_specie, 'biome')
            acronym_threat_category = get_atribute(df_specie, 'acronym_threat_category')
            lats_longs_animals = get_atribute(df_specie, 'lats_longs_animals')

            html_info = generate_species_info_html(comum_name, species, group, threat_category,
                                                    presence_in_protected_areas, national_action_plan_for_conservation_, 
                                                    fishing_planning, exclusive_species_of_brazil, occurrence_states, main_threats, biomes, acronym_threat_category)
            icon_path = f'images/test/{acronym_threat_category[0]}-{category[0]}.png'
            
            

            custom_icon = folium.CustomIcon(
                                        icon_image=icon_path,
                                        icon_size=(50, 50),  # Adjust the size of your icon as needed
                                        )
            
            yield (lats_longs_animals, html_info, custom_icon, category[0])
        except:
            pass



In [9]:
markers_tuples_generator = get_markers_tuples(df, df['simplified_species'].unique())

# Create a Folium map
m = folium.Map(location=[-14.235004, -51.92528], zoom_start=4)

layers_dict = {}
for category in df['category'].unique():
    layers_dict[category] = folium.FeatureGroup(name=category,show=False) 



# Add markers to the map
for lats_longs_animals, html_info, custom_icon, category in markers_tuples_generator:
    #print(lats_longs_animals, html_info, custom_icon, category, sep='\n')
    try:
        folium.Marker(location=lats_longs_animals, popup=html_info, icon=custom_icon, lazy=True).add_to(layers_dict[category])
    except:
        pass

for layer_name, layer in layers_dict.items():
    layer.add_to(m)


    
folium.LayerControl(collapsed=True).add_to(m)

Fullscreen(
        position="topright",
        title="Expand me",
        title_cancel="Exit me",
        force_separate_button=True,
        ).add_to(m)

folium.plugins.LocateControl(initialZoomLevel=3, auto_start=False, keepCurrentZoomLevel=True).add_to(m)

m.save('map_app.html')